# K-Means Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

We might want to take a look at the distribution of the Title+Body length feature we used before and instead of using the raw number of words create categories based on this length: short, longer,..., super long.

In the questions below I'll refer to length of the combined Title and Body fields as Description length (and by length we mean the number of words when the text is tokenized with pattern="\W").

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat,lit, col,udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import RegexTokenizer,VectorAssembler
from pyspark.ml.clustering import KMeans
# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read the dataset and extract description length features
# 3) write code to answer the quiz questions

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Description Length Features

In [8]:
df = df.withColumn("Desc", concat(col("Title"), lit(' '), col("Body")))

In [11]:
body_length=udf(lambda x: len(x), IntegerType())

In [14]:
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)
df = df.withColumn("DescLength", body_length(df.words))

In [16]:
assembler = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler.transform(df)

In [44]:
numtag=udf(lambda x: len(x.split(" ")), IntegerType())
df=df.withColumn("Numtags", numtag(df.Tags))

In [45]:
df.printSchema()

root
 |-- Body: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- oneTag: string (nullable = true)
 |-- Desc: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- DescLength: integer (nullable = true)
 |-- DescVec: vector (nullable = true)
 |-- Numtags: integer (nullable = true)



# Question 1
How many times greater is the Description Length of the longest question than the Description Length of the shortest question (rounded to the nearest whole number)?

Tip: Don't forget to import Spark SQL's aggregate functions that can operate on DataFrame columns.

In [22]:
# TODO: write your code to answer this question
df.createOrReplaceTempView('t1')
spark.sql('''
    SELECT round(MAX(DescLength)/MIN(DescLength),0)
    FROM T1''').show()

+-----------------------------------------------------------------------------+
|round((CAST(max(DescLength) AS DOUBLE) / CAST(min(DescLength) AS DOUBLE)), 0)|
+-----------------------------------------------------------------------------+
|                                                                        753.0|
+-----------------------------------------------------------------------------+



# Question 2
What is the mean and standard deviation of the Description length?

In [23]:
# TODO: write your code to answer this question
spark.sql('''
select avg(DescLength), std(DescLength)
from t1''').show()

+---------------+---------------------------------------+
|avg(DescLength)|stddev_samp(CAST(DescLength AS DOUBLE))|
+---------------+---------------------------------------+
|      180.28187|                     192.10819533505023|
+---------------+---------------------------------------+



# Question 3
Let's use K-means to create 5 clusters of Description lengths. Set the random seed to 42 and fit a 5-class K-means model on the Description length column (you can use KMeans().setParams(...) ). What length is the center of the cluster representing the longest questions?

In [46]:
# TODO: write your code to answer this question
kmeans=KMeans(featuresCol='DescVec',predictionCol="kmeans",k=5,seed=42)

In [48]:
model=kmeans.fit(df)

In [51]:
df=model.transform(df)

In [52]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Desc="How to check if an uploaded file is an image without mime type? <p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an imag

In [53]:
df.printSchema()

root
 |-- Body: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- oneTag: string (nullable = true)
 |-- Desc: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- DescLength: integer (nullable = true)
 |-- DescVec: vector (nullable = true)
 |-- Numtags: integer (nullable = true)
 |-- kmeans: integer (nullable = false)



In [54]:
center=model.clusterCenters()
for i in center:
    print(i)

[ 97.09383642]
[ 1077.93227792]
[ 502.39304611]
[ 2731.08284024]
[ 242.33112488]


In [61]:
df.createOrReplaceTempView('t2')
spark.sql('''
select kmeans, avg(DescLength), avg(Numtags), count(DescLength)
from t2
group by 1
order by 1''').show()

+------+------------------+------------------+-----------------+
|kmeans|   avg(DescLength)|      avg(Numtags)|count(DescLength)|
+------+------------------+------------------+-----------------+
|     0| 96.71484436347646|   2.7442441184785|            63674|
|     1|      1074.2109375|3.2864583333333335|             1152|
|     2|499.83863263173606|3.2294372294372296|             6699|
|     3|2731.0828402366865|  3.42603550295858|              169|
|     4| 241.0267434466191| 3.093549070868367|            28306|
+------+------------------+------------------+-----------------+

